In [20]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import time 
import pandas as pd
import re
import win32com.client as win32

servico = Service(ChromeDriverManager().install())

In [21]:
def busca_google_shopping(driver, nome_produto, termos_banidos, preco_min, preco_max):

    #TRATAR VALORES QUE VIERAM DA BASE DE DADOS
    nome_produto = nome_produto.lower()
    termos_nome_produto = nome_produto.split(' ')
    termos_banidos = termos_banidos.lower().split(' ')
   
    #ENTRAR NA GOOGLE STORE
    driver.get('https://shopping.google.com.br/')

    #SELECIONAR A BARRA DE PESQUISA E COLOCAR O NOME DO PRODUTO
    driver.find_element(By.CLASS_NAME, 'r7gAOb').click()
    driver.find_element(By.CLASS_NAME, 'r7gAOb').clear()
    driver.find_element(By.CLASS_NAME, 'r7gAOb').send_keys(nome_produto, Keys.ENTER)
    time.sleep(2)

    #SELECIONAR A DIV QUE POSSUI A LISTA DE TODOS OS PRODUTOS DA PÁGINA QUE NÃO ESTÃO SUJEITOS A JAVASCRIPT
    grid_procurar_produtos = driver.find_element(By.CLASS_NAME, 'BXIkFb')
    elementos_individuais = grid_procurar_produtos.find_elements(By.CLASS_NAME, 'sh-dgr__gr-auto')

    lista_ofertas = []#LISTA QUE VAI ARMAZENAR TODOS OS PRODUTOS QUE SERVEM

    #PASSAR PELA DIV DE CADA PRODUTO
    for resultado in elementos_individuais:

        preco_tag = resultado.find_element(By.CLASS_NAME, 'a8Pemb')
        preco_formatado = re.sub(r'[^0-9]', '', preco_tag.text)#TIRAR TUDO QUE NÃO SEJA UM NÚMERO DE 0 A 9
        preco = float(preco_formatado[:-2] + '.' + preco_formatado[-2:])

        tag_link = resultado.find_element(By.CLASS_NAME, 'Lq5OHe')
        titulo_tag = tag_link.find_element(By.TAG_NAME, 'h3')

        nome = titulo_tag.text.lower()
        link = tag_link.get_attribute('href')

        #VERIFICAR SE O NOME DO PRODUTO DO SITE TEM ALGUMA PALAVRA BANIDA
        tem_ban = False
        for palavra in termos_banidos:
            if palavra in nome:
                tem_ban = True

        #VERIFICAR SE TODAS AS PALAVRAS DO PRODUTO QUE ESTÃO NA BASE DE DADOS ESTÃO NO NOME DO PRODUTO QUE ESTÁ NO SITE
        tem_todos_termos = True
        for palavra in termos_nome_produto:
            if palavra not in nome_produto:
                tem_todos_termos = False

        if tem_ban == False and tem_todos_termos == True:
            try:
                if (preco_min <= preco) and (preco <= preco_max):
                    lista_ofertas.append((nome, preco, link))
            except:
                continue
            
    return lista_ofertas

In [22]:
def busca_buscape(driver, nome_produto, termos_banidos, preco_min, preco_max):
    
    #TRATAR VALORES QUE VIERAM DA BASE DE DADOS
    nome_produto = nome_produto.lower()
    termos_nome_produto = nome_produto.split(' ')
    termos_banidos = termos_banidos.lower().split(' ')

    #ENTRAR NO SITE BUSCAPE
    driver.get('https://www.buscape.com.br/?og=19220&og=19220&gclid=EAIaIQobChMIk6a9suSygwMVMlhIAB0SkgvtEAAYASAAEgImyPD_BwE')

    #SELECIONAR A BARRA DE PESQUISA E COLOCAR O NOME DO PRODUTO
    driver.find_element(By.CLASS_NAME, 'AutoCompleteStyle_input__WAC2Y').click()
    driver.find_element(By.CLASS_NAME, 'AutoCompleteStyle_input__WAC2Y').clear()
    driver.find_element(By.CLASS_NAME, 'AutoCompleteStyle_input__WAC2Y').send_keys(nome_produto, Keys.ENTER)
    time.sleep(2)

    #SELECIONAR O SELECT OPTION DA PÁGINA E COLOCAR PARA TER O MÁXIMO DE PRODUTOS NA PÁGINA
    qte_produtos = Select(driver.find_element(By.CLASS_NAME, 'HitsPerPage_Select__fLdHq').find_element(By.CLASS_NAME, 'Select_Select__1HNob'))
    qte_produtos.select_by_index(2)
    time.sleep(2)

    #SELECIONAR DIV QUE COMPORTA OS PRODUTOS
    grid_produtos = driver.find_element(By.CLASS_NAME, 'Hits_Wrapper__iA3rM')
    elementos_individuais = grid_produtos.find_elements(By.CLASS_NAME, 'Paper_Paper__4XALQ')

    lista_ofertas = []

    for div_produto in elementos_individuais:

        link = div_produto.find_element(By.CLASS_NAME, 'ProductCard_ProductCard_Inner__gapsh').get_attribute('href')
        
        nome = div_produto.find_element(By.TAG_NAME, 'h2').text.lower()

        preco_tag = div_produto.find_element(By.CLASS_NAME, 'Text_MobileHeadingS__HEz7L')
        preco_formatado = re.sub(r'[^0-9]', '', preco_tag.text)
        preco = float(preco_formatado[:-2] + '.' + preco_formatado[-2:])

        tem_ban = False
        for palavra in termos_banidos:
            if palavra in nome:
                tem_ban = True

        tem_todos_termos = True
        for palavra in termos_nome_produto:
            if palavra not in nome_produto:
                tem_todos_termos = False

        if tem_ban == False and tem_todos_termos == True:
            try:
                if (preco_min <= preco) and (preco <= preco_max):
                    lista_ofertas.append((nome, preco, link))
            except:
                continue

    return lista_ofertas

In [23]:
driver = webdriver.Chrome(service=servico)
driver.maximize_window()
df = pd.read_excel('buscas.xlsx')

tabela_ofertas = []

for linha in df.index:

    nome_produto = df.loc[linha, 'Nome']
    termos_banidos = df.loc[linha, "Termos banidos"]
    preco_min = float(df.loc[linha, 'Preço mínimo'])
    preco_max = float(df.loc[linha, 'Preço máximo'])

    lista_ofertas_google_shopping = busca_google_shopping(driver, nome_produto, termos_banidos, preco_min, preco_max)
    if lista_ofertas_google_shopping:
        tabela_google_shopping = pd.DataFrame(lista_ofertas_google_shopping, columns=['produto', 'preco', 'link'])
        tabela_ofertas.append(tabela_google_shopping)
    else:
        tabela_google_shopping = None
    
    lista_ofertas_buscape = busca_buscape(driver, nome_produto, termos_banidos, preco_min, preco_max)
    if lista_ofertas_buscape:
        tabela_buscape = pd.DataFrame(lista_ofertas_buscape, columns=['produto', 'preco', 'link'])
        tabela_ofertas.append(tabela_buscape)
    else:
        tabela_buscape = None

tabela_resultados = pd.concat(tabela_ofertas, ignore_index=True)
 


### Transformar a tabela de resultados numa tabela Excel

In [17]:
tabela_resultados.to_excel("Resultado da busca.xlsx", index=False)

### Enviar E-mail

In [19]:
if len(tabela_resultados.index) > 0:#ENVIAR SÓ SE HOUVER RESULTADOS
    # vou enviar email
    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.To = 'para quem será enviado'
    mail.Subject = 'Produto(s) Encontrado(s) na faixa de preço desejada'
    mail.HTMLBody = f"""
    <p>Prezados,</p>
    <p>Encontramos alguns produtos em oferta dentro da faixa de preço desejada. Segue tabela com detalhes</p>
    {tabela_resultados.to_html(index=False)}
    <p>Qualquer dúvida estou à disposição</p>
    <p>Att.,</p>
    """
    mail.Send()

driver.quit() #FECHAR NAVEGADOR